# Evaluation

### Library import

In [ ]:
from super_gradients.common.object_names import Models
from super_gradients.training import models

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_recall_curve, average_precision_score, PrecisionRecallDisplay, roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
from matplotlib import pyplot as plt

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

### Paths organization

In [ ]:
# Dataset
pwd_notebook = os.path.abspath('') # path notebook
root_path = os.path.dirname(os.path.dirname(pwd_notebook)) #path root project

data_dir = os.path.join(root_path, 'data') #path data

# train path
train_dir = os.path.join(data_dir, 'train')
print(train_dir)
# test path
test_dir = os.path.join(data_dir, 'test')
print(test_dir)
# valid path
valid_dir = os.path.join(data_dir, 'valid')
print(valid_dir)

In [ ]:
# Checkpoints
#Add absolute path of the checkpoint
checkpoint_path = 'C:\\User\\Documents\\brahma_project\\checkpoints\\arquitecture_distorsion\\resnet50_architecture_distorsion\\RUN_20240404_154343\\ckpt_best.pth'

n_classes = 4 #Add number of classes

#Add name of pre-trained weights
name_pretrained_weights = 'imagenet'

### load model

In [ ]:
model = models.get(model_name=Models.RESNET50, #Add model name
                   num_classes=n_classes, 
                   pretrained_weights=name_pretrained_weights, 
                   checkpoint_num_classes=n_classes, 
                   checkpoint_path=checkpoint_path)
model.to(device)

### Load Weights

In [ ]:
# Load the .pth file
checkpoint = torch.load(checkpoint_path)

# Check if the key 'state_dict' or some similar key exists
if 'state_dict' in checkpoint:
    model.load_state_dict(checkpoint['state_dict'])
elif 'net' in checkpoint:
    model.load_state_dict(checkpoint['net'])
else:
    # Whether the .pth file directly contains the model state
    model.load_state_dict(checkpoint)

model.to(device)

### Prepare test data

In [ ]:
# Defines transformations if they are not defined
test_transform = Compose([
    # Here are the same transformations that you used for training
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Load test data
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)

### Load Test DataLoader

In [ ]:
test_data = datasets.ImageFolder(root=test_dir, transform=test_transform)

test_dataloader = DataLoader(dataset=test_data, 
                             batch_size=16, # Adjust as needed
                             num_workers=2, # Adjust based on your available resources
                             shuffle=False)

### Inference

In [ ]:
# Get an image from the test set (replace 'index' with the index of the image you want)
index = 2  # Change this to the desired image index
image, true_label = test_data[index]  # Get the image and its true label

# Perform model inference on the image
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    image = image.unsqueeze(0)  # Add a batch dimension since the model expects batches
    output = model(image.to(device))
     # Convert logits to probabilities
    probabilities = F.softmax(output, dim=1)
    _, predicted_label = torch.max(output, 1)  # Get the predicted label
    predicted_score = probabilities[0][predicted_label.item()].item()  # Get the probability of the predicted label

# Get the name of the predicted class from the index
predicted_class = test_data.classes[predicted_label.item()]
true_class = test_data.classes[true_label]


# Show image along with predicted and true label
plt.imshow(image.squeeze().permute(1, 2, 0))  # Show image (make sure dimensions are appropriate)
plt.title(f"Predicted: {predicted_class} \n Score: {predicted_score*100:.2f}% \nTrue: {true_class}")
plt.axis('off')
plt.show()

### Show Df inference

In [ ]:
pd.set_option('display.max_colwidth', 1000)
# Create table with the results de test (true, predict, score and path image)
results = []
for i, (image, true_label) in enumerate(test_data):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        image = image.unsqueeze(0)  # Add a batch dimension since the model expects batches
        output = model(image.to(device))
        # Convert logits to probabilities
        probabilities = F.softmax(output, dim=1)
        _, predicted_label = torch.max(output, 1)  # Get the predicted label
        predicted_score = probabilities[0][predicted_label.item()].item()  # Get the probability of the predicted label
        # pass predicted_score to percentage with 2 decimals
        predicted_score = "{:.2f}".format(predicted_score*100)
    # Get the name of the predicted class from the index
    predicted_class = test_data.classes[predicted_label.item()]
    true_class = test_data.classes[true_label]
        
    # Add new column with "X" where the true class is equal to the predicted class
    if true_class == predicted_class:
        true_true = ' (X)'
    else:
        true_true = ' ( )'
    
    # in the path image add name folder and image and not path complete
    # results.append([true_class, predicted_class, predicted_score, test_data.imgs[i][0].split('\\')[-2] + '/' + test_data.imgs[i][0].split('\\')[-1]])    
    
    # in the path image add path complete
    results.append([true_class, predicted_class, predicted_score, test_data.imgs[i][0], true_true])

# Create a DataFrame from the results
results_df = pd.DataFrame(results, columns=['True', 'Predicted', 'Score', 'Image Path', 'Acertado'])

results_df

### Accuracy and Classification report

In [ ]:
all_preds = []
all_labels = []

# Evaluate the model on the test data set
model.eval()
with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate general metrics
accuracy = accuracy_score(all_labels, all_preds)
conf_matrix = confusion_matrix(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=test_data.classes)

print(f"\n Accuracy: {accuracy}\n")
print("Classification Report:\n", report)

### Confusion Matrix

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='g', xticklabels=test_data.classes, yticklabels=test_data.classes, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

### Precision-Recall Multiclass

In [ ]:
#Obtain Probabilities of each Class
y_true = []
y_scores = []

model.eval()
with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1).cpu().numpy()

        y_true.extend(labels.cpu().numpy())
        y_scores.extend(probabilities)

y_true = np.array(y_true)
y_scores = np.array(y_scores)

# Binarize tags
y_true_bin = label_binarize(y_true, classes=np.arange(len(test_data.classes)))
n_classes = len(test_data.classes)

# Calculate average precision and recall for each class
precision = dict()
recall = dict()
average_precision = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_scores[:, i])
    average_precision[i] = average_precision_score(y_true_bin[:, i], y_scores[:, i])

# Colors for different classes
colors = cycle(['blue', 'red', 'green', 'yellow', 'orange', 'purple'])

# Draw Precision-Recall curves and iso-F1 curves
plt.figure(figsize=(7, 8))
f_scores = np.linspace(0.2, 0.8, num=6)
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    # l, = plt.plot(x[y >= 0], y[y >= 0], color="gray", alpha=0.2) # iso-F1 curves
    plt.annotate(f"f1={f_score:0.1f}", xy=(0.9, y[45] + 0.02))

for i, color in zip(range(n_classes), colors):
    plt.plot(recall[i], precision[i], color=color, lw=2,
             label=f'Precision-Recall curve of {test_data.classes[i]} (AP = {average_precision[i]:0.2f})')

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="best")
plt.title("Precision-Recall curve to multi-class")
plt.show()

### ROC Curve || FPR vs TNR

In [ ]:
# Binarize tags
y_true_bin = label_binarize(y_true, classes=np.arange(len(test_data.classes)))
n_classes = len(test_data.classes)

# Calculate ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Colors for different classes
colors = cycle(['blue', 'red', 'green', 'yellow', 'orange', 'purple'])

# Draw the ROC curve for each class
plt.figure(figsize=(10, 8))
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'ROC curve of {test_data.classes[i]} (area = {roc_auc[i]:0.2f})')

# plt.plot([0, 1], [0, 1], 'k--', lw=2)  # Draw line for classifier
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (Fall out)')
plt.ylabel('True Positive Rate (Sensivity)')
plt.title('ROC Curve for Multi-Class FPR vs TPR')
plt.legend(loc="lower right")
plt.show()

### ROC Curve || Specificty vs Sensibility

In [ ]:
# Binarize tags
y_true_bin = label_binarize(y_true, classes=np.arange(len(test_data.classes)))
n_classes = len(test_data.classes)

# Colors for different classes
colors = cycle(['blue', 'red', 'green', 'yellow', 'orange', 'purple'])

plt.figure(figsize=(10, 8))

# Calculate ROC curve (TNR and TPR) and ROC area (AROC) for each class
for i, color in zip(range(n_classes), colors):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
    tnr = 1 - fpr
    roc_auc = auc(tnr, tpr)

    plt.plot(tnr, tpr, color=color, lw=2, label=f'{test_data.classes[i]} (AROC = {roc_auc:.2f})')

plt.xlabel('True Negatives Rate (TNR - Specificity)')
plt.ylabel('True Positive Rate (TPR - Sensibility)')
plt.title('ROC Curve for Multi-Class Specificity vs Sensibility')
plt.legend(loc="best")
plt.show()